In [1]:
import os,sys

In [2]:
os.chdir('../')


In [3]:
pwd

'e:\\gemstone'

In [4]:
import pandas as pd
import numpy as np
from src.utils.utils import read_yaml,create_dir,save_obj,model_evaluate
from src.constant.ymal_path import *
import yaml
from dataclasses import dataclass

In [5]:
from pathlib import Path

@dataclass
class ModelTrainExConfig:
    dir:Path
    train_arr:Path
    test_arr:Path
    model: Path
 

In [6]:
class ConfigManager:
    def __init__(self,
                    config_filr_path=Config_ymal_file_path,
                    models_file_path=Model_ymal_file_path):
            self.config=read_yaml(config_filr_path)
            self.models=read_yaml(models_file_path)
          
           

            create_dir([self.config.artifacts_root])

    def get_model_train_config(self):
          config=self.config.model_trainer_exp
       

          create_dir([config.dir])

          model_train_config=ModelTrainExConfig(
                dir=config.dir,
                train_arr=config.train_arr,
                test_arr=config.test_arr,
                model=config.model
              
               
                
          )
          return model_train_config

In [7]:
from src.logging.logger import logging
from src.exception.exception import CustomException
from sklearn.linear_model import LinearRegression,Lasso,ElasticNet,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [18]:
def model_evaluate(x_train, y_train, x_test, y_test, models):
    trained_models = {}

    # Train and evaluate each model
    for model_name, model_params in models.items():
        model = None
        if model_name == 'DecisionTree Regressor':
            model = DecisionTreeRegressor(**model_params)
        elif model_name == 'ElasticNet Regression':
            model = ElasticNet(**model_params)
        elif model_name == 'Lasso Regression':
            model = Lasso(**model_params)
        elif model_name == 'Linear Regression':
            model = LinearRegression(**model_params)
        elif model_name == 'RandomForest Regressor':
            model = RandomForestRegressor(**model_params)
        elif model_name == 'Ridge Regression':
            model = Ridge(**model_params)
        else:
            print(f"Unknown model type: {model_name}")
            continue
        
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)

        accuracy=r2_score(y_test,y_pred)
        mae = mean_absolute_error(y_test, y_pred)
       

            # Calculate mean squared error
        mse = mean_squared_error(y_test, y_pred)
          
        trained_models=[
                f'Accuracy: {accuracy:.2f}%  MAE: {mae:.2f}%  MSE: {mse:.2f}%'   
            ]
    return trained_models


In [22]:
from mlflow import models


class ModelTrain:
    def __init__(self,config:ModelTrainExConfig) -> None:
        self.config=config

    def initate_model_trainer(self):
        try:
            train_array=np.load(self.config.train_arr)
            test_array=np.load(self.config.test_arr)

            x_train, y_train, x_test, y_test = (
                train_array[:,:-1],
                train_array[:,-1],
                test_array[:,:-1],
                test_array[:,-1])
            

            model_report:dict=model_evaluate(x_train, y_train, x_test, y_test,models=self.config.model)
            

            print( model_report)

            
            logging.info(f'Model Report : {model_report}')

            print('\n====================================================================================\n')

            # 6. Find the best model
            best_model_score=max(sorted(model_report.values()))

            
            best_model_name = list(model_report.keys())[
                list(model_report.values()).index(best_model_score)
            ]

            best_model = models[best_model_name]

            print(f'Best Model Found , Model Name : {best_model_name} , Score : {best_model_score}')
            print('\n====================================================================================\n')
            logging.info(f'Best Model Found , Model Name : {best_model_name} , Score : {best_model_score}')


            save_obj(
                file_path=self.config.model,
                obj=best_model
            )
            return best_model
            
            
            
        except CustomException as e:
            logging.info(f'Error cooured {str(e)}')
            raise CustomException(sys,e)    


In [23]:
import yaml

# Replace 'your_file.yaml' with the path to your YAML file
with open('models.yaml', 'r') as file:
   models= yaml.safe_load(file)



In [24]:
try:
    config=ConfigManager()
    model_train_config=config.get_model_train_config()
    model_train=ModelTrain(config=model_train_config)
    model_train.initate_model_trainer()
except CustomException as e:
            logging.info(f'Error cooured {str(e)}')
            raise CustomException(sys,e)       

AttributeError: 'str' object has no attribute 'items'